In [1]:
import pandas as pd
import yfinance as yf
import requests
import time
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client.db

In [3]:
threaddf = pd.DataFrame.from_records(db.threads.find({'created_utc': {'$exists': 'true'}}))

In [6]:
threaddf['comments'] = 0
threaddf['comments'] = threaddf['comments'].astype('object')

In [7]:
def getData(datatype, **kwargs):
    url = f'https://api.pushshift.io/reddit/search/{datatype}/'
    payload = kwargs
    req = requests.get(url, params = payload)
    return req.json()['data']

def getIds(id):
    url = f'https://api.pushshift.io/reddit/submission/comment_ids/{id}'
    req = requests.get(url)
    return req.json()['data']

In [21]:
'''for i in range(356, len(threaddf)):
    print(i)
    threaddf.at[i,'comments'] = getIds(threaddf.loc[i,'id'])
    time.sleep(1)'''

356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590


In [22]:
threaddf.loc[threaddf['comments'] != 0]

,_id,id,created_utc,title,num_comments,score,comments
0,5fab839d169c4ce5e3a76030,dbrref,1569927961,"Daily Discussion Thread - October 01, 2019",6135,58,"[f23jn52, f23jn90, f23jo5k, f23jolu, f23jpa1, ..."
1,5fab839d169c4ce5e3a76031,dbzt61,1569963883,"What Are Your Moves Tomorrow, October 02",3056,50,"[f2514mp, f25178m, f25184t, f251b4p, f251cbf, ..."
2,5fab839d169c4ce5e3a76032,dc8o17,1570014357,"Daily Discussion Thread - October 02, 2019",8300,79,"[f26kb1w, f26kbkq, f26kc2r, f26kc73, f26kfb9, ..."
3,5fab839d169c4ce5e3a76033,dcgisg,1570050273,"What Are Your Moves Tomorrow, October 03",2753,54,"[f280zzq, f281062, f2810aq, f2810cg, f2810t8, ..."
4,5fab839d169c4ce5e3a76034,dcpbm1,1570100727,"Daily Discussion Thread - October 03, 2019",7500,69,"[f29mhgv, f29mjvc, f29mkak, f29mmug, f29mr5f, ..."
...,...,...,...,...,...,...,...
586,5fab839d169c4ce5e3a7627a,j1jqk5,1601323620,"What Are Your Moves Tomorrow, September 29, 2020",14483,1,"[g6zmz4a, g6zmz4i, g6zmz86, g6zmze0, g6zmzed, ..."
587,5fab839d169c4ce5e3a7627b,j1whna,1601374331,"Daily Discussion Thread for September 29, 2020",15666,1,"[g71nrlp, g71nrmp, g71ns8b, g71nsi7, g71nska, ..."
588,5fab839d169c4ce5e3a7627c,j272rv,1601409994,"What Are Your Moves Tomorrow, September 30, 2020",26649,1,"[g73lr0q, g73lr7t, g73lrci, g73lrey, g73lrg3, ..."
589,5fab839d169c4ce5e3a7627d,j2jg0e,1601460698,"Daily Discussion Thread for September 30, 2020",20353,1,"[g75pp9u, g75pp9x, g75ppb2, g75ppla, g75ppyp, ..."


In [24]:
len(threaddf.loc[356,'comments'])

18463

In [25]:
threaddf.loc[356,'num_comments']

19427

In [26]:
threaddf['numComments'] = len(threaddf['comments'])

In [28]:
for i in range(len(threaddf)):
    threaddf.at[i, 'numComments'] = len(threaddf.loc[i, 'comments'])

In [29]:
threaddf['numComments']

0       6135
1       3056
2       8300
3       2753
4       7499
       ...  
586    14477
587    15663
588    26643
589    20349
590    12690
Name: numComments, Length: 591, dtype: int64

In [33]:
'''bulk_update = db.threads.initialize_unordered_bulk_op()
for index, row in threaddf.iterrows():
    _id = row['_id']
    commentArray = row['comments']
    commentAmount = row['numComments']
    bulk_update.find({'_id': _id}).update_one({'$set':{'comments': commentArray, 'numComments': commentAmount}})
bulk_update.execute()'''


"bulk_update = db.threads.initialize_unordered_bulk_op()\nfor index, row in threaddf.iterrows():\n    _id = row['_id']\n    commentArray = row['comments']\n    commentAmount = row['numComments']\n    bulk_update.find({'_id': _id}).update_one({'$set':{'comments': commentArray, 'numComments': commentAmount}})\nbulk_update.execute()"

In [ ]:
threaddf['comments']

In [5]:
'''subreddit = 'wallstreetbets'
lastoctober = 1569888000
thisoctober = 1601510400
size = 500'''

In [7]:
'''
df = pd.DataFrame()
after = lastoctober
before = thisoctober
while before > after:
    testdict = getData(datatype = 'submission', subreddit = subreddit, after = after, q = 'Daily Discussion Thread', before = before, size = size)
    if not testdict:
        break
    tempdf = pd.DataFrame.from_records(testdict)
    df = df.append(tempdf, ignore_index = True)
    after = df['created_utc'][len(df)-1]
    print(after)
    time.sleep(1)'''

1575630325
1581368695
1586370524
1591733184
1596829613
1601496415


In [34]:
threaddf['numComments'].sum()

8835137

In [40]:
import random
def pareComments(comments):
    random.shuffle(comments)
    return comments[0:int(len(comments)*0.3)]

In [42]:
len(pareComments(threaddf.loc[0,'comments']))

1840

In [44]:
threaddf['pareComments'] = threaddf['comments'].apply(pareComments)

In [46]:
for i in range(len(threaddf)):
    threaddf.at[i,'pareTotal'] = len(threaddf.loc[i, 'pareComments'])

In [47]:
threaddf

,_id,id,created_utc,title,num_comments,score,comments,numComments,pareComments,pareTotal
0,5fab839d169c4ce5e3a76030,dbrref,1569927961,"Daily Discussion Thread - October 01, 2019",6135,58,"[f24h9o9, f23y8fn, f24vyaa, f24dxww, f243mu5, ...",6135,"[f24h9o9, f23y8fn, f24vyaa, f24dxww, f243mu5, ...",1840.0
1,5fab839d169c4ce5e3a76031,dbzt61,1569963883,"What Are Your Moves Tomorrow, October 02",3056,50,"[f260skw, f26kftw, f25s72n, f25glny, f25t39n, ...",3056,"[f260skw, f26kftw, f25s72n, f25glny, f25t39n, ...",916.0
2,5fab839d169c4ce5e3a76032,dc8o17,1570014357,"Daily Discussion Thread - October 02, 2019",8300,79,"[f26t5w3, f27v5ub, f27guz6, f27g3lj, f27usrp, ...",8300,"[f26t5w3, f27v5ub, f27guz6, f27g3lj, f27usrp, ...",2490.0
3,5fab839d169c4ce5e3a76033,dcgisg,1570050273,"What Are Your Moves Tomorrow, October 03",2753,54,"[f28p09e, f28t4yt, f28da1p, f293m5i, f29bg6c, ...",2753,"[f28p09e, f28t4yt, f28da1p, f293m5i, f29bg6c, ...",825.0
4,5fab839d169c4ce5e3a76034,dcpbm1,1570100727,"Daily Discussion Thread - October 03, 2019",7500,69,"[f2ahcpj, f2b0pg2, f2c4uu5, f2aekop, f2bbzlk, ...",7499,"[f2ahcpj, f2b0pg2, f2c4uu5, f2aekop, f2bbzlk, ...",2249.0
...,...,...,...,...,...,...,...,...,...,...
586,5fab839d169c4ce5e3a7627a,j1jqk5,1601323620,"What Are Your Moves Tomorrow, September 29, 2020",14483,1,"[g70yxs5, g71234w, g6zvqiu, g703lof, g714foz, ...",14477,"[g70yxs5, g71234w, g6zvqiu, g703lof, g714foz, ...",4343.0
587,5fab839d169c4ce5e3a7627b,j1whna,1601374331,"Daily Discussion Thread for September 29, 2020",15666,1,"[g71rs8f, g72rll0, g72ji7v, g71s43g, g7387kt, ...",15663,"[g71rs8f, g72rll0, g72ji7v, g71s43g, g7387kt, ...",4698.0
588,5fab839d169c4ce5e3a7627c,j272rv,1601409994,"What Are Your Moves Tomorrow, September 30, 2020",26649,1,"[g74xwko, g74gq2t, g745koy, g74np3c, g75n21f, ...",26643,"[g74xwko, g74gq2t, g745koy, g74np3c, g75n21f, ...",7992.0
589,5fab839d169c4ce5e3a7627d,j2jg0e,1601460698,"Daily Discussion Thread for September 30, 2020",20353,1,"[g76hxe2, g76nbtk, g77firr, g75r3kt, g766v4u, ...",20349,"[g76hxe2, g76nbtk, g77firr, g75r3kt, g766v4u, ...",6104.0


In [50]:
commentList = threaddf.loc[0,'pareComments']

In [51]:
import praw

In [53]:
secret = db.secret.find_one()['Secret']
reddit = praw.Reddit(client_id='-wpcPIbA7bhlpw', client_secret=secret, user_agent='sentiment')

In [56]:
commentlist = ['t1_' + i for i in commentlist]

In [92]:
commentdf = pd.DataFrame(columns = ['id', 'created_utc', 'body', 'score'])

In [89]:
for i in range(241, len(threaddf)):
    print(i)
    commentlist = threaddf.loc[i,'pareComments']
    commentlist = ['t1_' + x for x in commentlist]
    total = threaddf.loc[i, 'pareTotal']
    counter = 0
    for comment in reddit.info(commentlist):
        commentdf = commentdf.append({'id': comment.id, 'created_utc': comment.created_utc, 'body': comment.body, 'score': comment.score}, ignore_index = True)
        if total < 10:
            continue
        elif counter % (int(total / 10)) == 0:
            print('\t' + str(counter) + '/' + str(total))
        counter += 1

46.0
	192/646.0
	256/646.0
	320/646.0
	384/646.0
	448/646.0
	512/646.0
	576/646.0
	640/646.0
81
	0/1153.0
	115/1153.0
	230/1153.0
	345/1153.0
	460/1153.0
	575/1153.0
	690/1153.0
	805/1153.0
	920/1153.0
	1035/1153.0
	1150/1153.0
82
	0/822.0
	82/822.0
	164/822.0
	246/822.0
	328/822.0
	410/822.0
	492/822.0
	574/822.0
	656/822.0
	738/822.0
	820/822.0
83
	0/1250.0
	125/1250.0
	250/1250.0
	375/1250.0
	500/1250.0
	625/1250.0
	750/1250.0
	875/1250.0
	1000/1250.0
	1125/1250.0
84
	0/867.0
	86/867.0
	172/867.0
	258/867.0
	344/867.0
	430/867.0
	516/867.0
	602/867.0
	688/867.0
	774/867.0
	860/867.0
85
	0/1244.0
	124/1244.0
	248/1244.0
	372/1244.0
	496/1244.0
	620/1244.0
	744/1244.0
	868/1244.0
	992/1244.0
	1116/1244.0
	1240/1244.0
86
	0/638.0
	63/638.0
	126/638.0
	189/638.0
	252/638.0
	315/638.0
	378/638.0
	441/638.0
	504/638.0
	567/638.0
	630/638.0
87
	0/243.0
	24/243.0
	48/243.0
	72/243.0
	96/243.0
	120/243.0
	144/243.0
	168/243.0
	192/243.0
	216/243.0
	240/243.0
88
	0/489.0
	48/489.0
	96/489.0
	

KeyboardInterrupt: 

In [90]:
commentdf

,id,created_utc,body,score
0,f3azvn7,1.570798e+09,To ease your concerns: hello Government agenci...,1
1,f3b0wgn,1.570799e+09,This would never work\nHaha it just cant\nUnle...,1
2,f3nygp2,1.571004e+09,And the sheeple rejoice. China literally bough...,4
3,f3p0fzf,1.571037e+09,Me too comrade,1
4,f3o2twv,1.571007e+09,a few far expiration OTM puts and SPY/AAPL calls,1
...,...,...,...,...
441482,fjmnvm4,1.583477e+09,Fuck someone in my fucking neighborhood just w...,17
441483,fjlaxlf,1.583443e+09,That 205 put lmfao wtf,32
441484,fjmvumh,1.583487e+09,"China down 2%? Pack it up boys, even FED pumpi...",4
441485,fjltjoq,1.583454e+09,vix futures down.. jomentum here we come,4


In [91]:
db.comments.insert_many(commentdf.to_dict('records'))